In [2]:
import requests
import p03_wiki
import os
import json
import base64
import glob
import time
import gzip
import functools

In [8]:
datastr = 'H4sIAAAAAAAAC+VdbVMbRxKez1d1/+FOdd+SUOFFgO+bBALbQUCQbJK4XESAAB0SIiuBjZP89+t+ZlYvq1lpZneF20OpbImVdnamn3npp7un50/1T/UP9S/zKqn/qI66pPf/0l8/qu9nvh2qJ3Wv2uYXJfrNX2qX7hmoW7WjWiqiu/fpvUe/WVEVun5Bn+7ovaP69N6g7x7pyi69D+nf91QK/2pA13rqXHWp/B+opIa6QWn39IxkLS6prBb9sq+u1cNEWbpOf0792t6y1USZ81q4OreFu4m61NC6OyplkKON4/q0R+W5tNLe2rWU0ue1eo1a3aBrA/qGa71C9e1Tzbv01wVd02gO6Po+6temdnTomxV1AFkN1e8kuQ/0yie77yau+vedj/TiX/XQD/t0rUvPPE+V96REIrT1juo2pPLifrNOL5d7LyCfoam5vvvD3Pvi13xk5yO9qG4uyK/n7u/5MFuEzbwx8mYkiZI6prlpXe1Rfe9QY35Cn8o/pP8v0V7f50R0F0uKW8Kj4WEC21V6uZT2t+NT8/SCjdy9wL8fzcNgdhU4hizf4PcRXSkWi7IYJMrikDiHrIaqXvgIkCP1TdFSXwtW7lui5b4erNy3hcl9PK9fksTG2mKT/q/j6iVW5etgEXklDJFjaFescx1BVw9V7qsznPVrSz7WOu9Nj98DC9GjoFgU1gWhkMavvx4K7RFzbdO3oct/EeN/bvmP55+m+lS4vi9J8vkZ+HIlz2vzjrFNRLDohIuFNB6cxOKUPnVgX4ntLIOA0ZDGhZNoVDEq2oVjIMcutCqOGU9icAMpFq2bShoB0vixTfoNWGkjWG5Zc/1k9NYjdUWvcLGRxqFt2FTp3w3W7JB1V7nceXKUxHc04Ath78dqsJisiWPVsx6FXfrrykiyXfj42BCEhTRuXTUyG3vaWJMaFK5HScJAGr8+xBx0Y9bqOuTH4yAKGgVpXHu8PrB96RFcokLXWJIhz0hyePYBZH5DZc/ORsnIhWLxWBOEhxym/RooDNWZmaFq9BfL7ZowqpmooWt6crhjQxrjbpCkIkjvDlEwUcCyl8O3OX6KRwJ7pDuIQePn7IDN9cEtYgtgsWhsCUJDDsO2o9E0+tPtUnzTkpCQw7DTxkVf9ehp7LFbBpuTg8W6IGbdJJm1sCaMvRFF2zRWRcleDpO2yb7oODFZspfDoF/DotqnOSiC5E6pnl8MCk3VHs1Ey7FqyLHyLY5sfz5MeBTc0Tdsd11urLAk+cvh0ZPyn9RST0h2PBbCtXWvC2LPi1AoeoWQhIIc3jw9F8W+oLUJT2molqR1QQx6bEnaMTvPHmhlbmEHEe8uesD+M+Ztu9ClPk1gFWo8x7ogTp3Uot4Z+1LrWbQoSZqtHHZdwT5Etmo8mj2Zy9Wl5KCwIYhXz6LwEuQvh1vPyv8Ae4AjrN4drO672N/aotWjTXUPFxU5rHsWFdsu0WVGjW8KwkUO87atGX28LzfCRhIacnh4dSSlJ9Kw9Cf22Y1XENazerCTF83J5fCQDUGcPA2Rn+C/6GJkhIyEHF4eR5/Z8ahifWdGGCr72xDEzmex+BkxNsvUduVYqzYE8fB5SIwtVsWv35LQkMPAv2C3L/f9FcxFrOFGeFK8J15fv8L/Hev390D0lqT+BP2YZzeW/BlxGB3n2YV9pW921euxdqbqKG+AdSnkXU1lQVx/dvQ1MeKuoaHVRqtT0WjI2YFfFsT856PBdn0diVi0BV/OXFgWxPjnrUzJiJ9wR4ccpn9sdIF10ptbsNbvkNSuwPGL1g4krRZy2P3Yp3VMPZ/X9+f1Y0lCRQ7DT9thdqI4fwjrYN3CsZDD8MviGP4sEpxH6hZXis7pJWlESGL38Uphs9G3UOsTrOT3mLHCtbmUBTH9Ksmog73glVHU6LJsLZIQkMPubQgsV5OVg8OmINZtw6FB81Qr4JloUxDPtsn/2Oixy7A1SkJBGr+epy0Vv6NADg6SeLXeC87RujUwt0lLR7gIyOHVNgQqNCouIMnQZyQ5TPqQkNgx8dN32P+9j7gSjkDUp0qEanfdFMeij5HLi30PNn9guKNBEotOw+HlzE1y2HMSjeboU/F7XuXIXw53Tpd/uFrqliDOnC7/cG2qW4I4c7r8i44/lyR/aWz5FJHOvOruwQN3jcyOQ5JflnOovhUUpHHltAiNA8hS51EL1d+2JY43nyJjnd4nNiDGxiz6HFfC3em6JYg1axSSuaHiHch6F0YLO8DD5W1b4vhzHZ7/vskfqHfGnOJsqVBtGFuCuLNd+m3suAg/vnxLEG+2IXGEFbuFCOYLZBjU9ozi93lLWjHkcOnFmOxBnkXnepSDxrYgZu2CRgv+0gdIN9Q1fFsQ27ZhMt4h9jJmrG1B7NuGx9gW8lLwkMPDbXjUjF71BM7BM1fIvoltcTy8gihwln+MzjLXDUnjQg4Xnz8uKuAfetYKeWTI4eI2PNhiyPFNyzjZVtKokM3G48iOKsnxgvAIFwc5XDy5Tkyed64RWk62NUlzk2wWPhuNGToir4Qz8cEIAR2pfw4GGO668Uo4C4/3Fo1PJw57/8qrb4aFsz7VXcrokISGbA4+tlG9DDTkcHA704jjZh9o3fif4v3Aw6DXctlMPA2PJnbM80k+IWMjm5U3kJ+7B51Xx5CEi4QcZq4lrfc+RrBKvRQm+EoQL7eNh9dKx1Rx1MJ+0KNBDiPXo+GIVgNGhP8K10q7SrxbDvPWkj+llXgvaInL4daTEg93BwXLXDZ/TuZ/WKYvWxYucph0HMfMY2EXtfwVktsNeN1lBOSwZ43Ar+j7RfuCZMlcDkPWMn+Nkyx60Dbr0Hj4yj29ij5lWxYOcthwHC+uI/S1VzRCHP+yrNtyvA2MhBw2fIBV9wYRNC8VDTmsOH1cNGi+eij8xG1ZOMhhxYtxOMWeyBt1iLUkXN/oKpD5dnCZ9pWGjIocXj1/DTkGz1tGdl5ZeMjh3OmjRJ8Leme8QOHquqvimPZiNJYxY8nCRBr3tmFSQzRayzwzXCykcfJFWIQ/OqSx830VqfgkQ869EiFeII7VvMT/F4GPEjk8fR+5YjlWIFL6lPVu0Ll7WfpyePn4FIr4xInJUyj26D0y0TZ8NWxUpLH0ZWSYkCTxNXH8e89I6zKR+aYJTfamYCzk5JhgLOSwbo3FCUnrieTWHUUvhR8fzjjIYdt15ETjvb9XyFzwQBjYtFmOU36AnSRkXOTw7mmNyYbIzyYnfzdoRKSx7gOlM3a9w5gZo1EJGgVpfDsNhWrQKEjj2Gko7ASNghxWPR+F3aBRkMOuY3sTRwMOcO5ByHKXxp9fG1kNwOuihN4ULhLrgnj1Yl11D7am1hLO1ZSFiRx+7crrXgoychj3/HiDl4LHt8S0JzEJeefFuiC2nWX+ChsbORzcfQb7WojM+9VH67fT9yQxLVGNIuwBipTe56Nz/eyili1TOzvCs0im8fhZxEo0Iv/CDpeBuh2dvLlvojZXcAZneq2+pxIquN5GHpwulf0DldLA7vlIpZ+AUTJRbxHiey6Rd6qD7BRDZIkczG1xessXcWebBMokgQZdGyh9otwKekUXNbxAffrw7K+Ag92hH/KMsUL9VO/Y+p2e/YFe6XKcbN+Tg9w+0ot/1UNEAa8cfO/5wt7O7YtQc/a8DmFN1yPTRTIRnnUHOYzHxAenMeHu+XTPzO9+poK7ZuzOMN3tYe7Pdm2N2+9cV1NXKbrj4vY7VxuKKyLusdbu8b/ukanuUZPusXzukWbuEVDucTnuMSTuXnZ3H7C7l9Ldb+buz3H3OZSdn172GJGubS87y9P9JGmfnf56v3QWfcdt5a8rHRURZ7bPuvIv6sm2lX/zGVb+2fbpuOb4tM7n1wUWIZ9VE8jXD2LJZcV/0Rpiw3+rMPynS2kgwkRHv9mQKxpRF2ttXv2upGrISsAyfG/YAJ+ec4aTKVqoURtycj+TuaT+rap0/zt1qJrqV/XafKqpEyq3SuOiQZ/dS6uafJycw9bdS1LCWOugNZfe99bRY9qwJ7hHNpaQEb9Fz2uCW7nb4krUR7vGIxehxT61rYLP3QJ/33u1P+Qnen6d0Nkjxn5C6JwSSgf0ci8nrR9VkZfevw/p358QgvcY3e5n1JSQc/pKDb1l0cBs7vu0hhl3Ogdj18SQt736TQWWxHaGPv4JewXyoP4L/bVKSDWQC+DexFj79N0qRgvnsopjJLPXZI1qUje5U4b07zEDEpzz2lciaf13jKZvD7a1rGYyMLQwKzXUH4jG8pFXBZG/PCO28O7Ty3QLauozUO5jP4NPTz/AMy/MOZCT82s+qewjul9rUJeQSN6+fIoedJV7XMxitk/10zpNx3NtqMIarU9yPzaaxNATAVv9qliptCZTMXmHnlCye7lvoBFp/Yj7RQQsfEqwS05rV9cYVS2MsI7yOaOkZM56LWaG02ssZ57xnWdtpVVMHkvfmk2Poip6wpBePr67EmKguRUNrPs6q5TfbJCsh9ay8iGuT0vuGS2ZZ5o2GFS2Gv5CI5j16J6pYSvDmt407WqYp/v1PdbfI4x339E+rVe9h171jmrB6+wO9Z1DerFGnLXEI6rdLvbmVKiN7nGk9r58BDv2W8g58sgIZC/tNxVn0G8bX4NvnrPF5e4DF30GtD+y07KcZCtVwqRCaB16sQWt++2YPAw+GssQ81sv14w03QLuDScknxp9yidvPX4vjMbhs07F/OcMOmHXeJtaXvrTTmL8V2hd8tVYbK16P9Lxi1hT9gxPzsa+0uV0BjbCOfd3oAXrLLFZNa2zGWme0FPOMZay97pkmUckV70f5yGzpmyraewnbsBD7M8HbW1/9GRoyTJiDdO9hLfgRh2U0UCcgt96H3vVW8bKk6fXniAz+hP1vw58vllXtnjdP6ayogy6kK2sWLMZAqe8jCtmyLrPTOs8eUu+wFjNr5tqCfaheftpgS5a7hnyW37KjXFR2lBc3lEGyWkdULMmf45fwb36HLKsbHP6vqaKT0XxKUGfT8588MprBklammLbWtaZdp/GQR+xOJ9MXvisuvkxPrOmcG9q54tNLJXYdpe1TWw7+kx96xzlaf9N1rKOqZxTY7dujj7ntXXo1axCpbUya9qzqySfOXkP75I/h6zD/5GX7Wv/SBs8r4FR0fZeqW3lvkWcZY9k9bYAWwn3jmL0PrayaJ5b5KpybKKJGpDfrWeJFfBv7hP5/AB14hE71ONZL63BF8CMaB/97og+7RLbeJOLX6zBghJhNtO+OX2iMGsRfhbRtJrrmjaolsy8d+g9i/cpza6lc/QMMHu2vPmRfT0ce9z0LHY5shjmLfu0sNH4DvP8DXp+PGfnnTcezPzjV5Y+1fIRvJszkDzmnsG49/2Wm43HM0TdaOodc7agX3kR9JShkUzdmy8kR8VsxG3s4yzKV3jmEbe7uKzVAsv6MbdGzTofS+4J47yI+f7MWLBYl7xCebq0sfaed47qYsTfFMIX9Wp0WZCfsUYv932m8zCJ9fBd+E61Pq29Dr1cVsw1rHx6RroyPossa9NrWPe6Btni/FvTZWkb63Dkv8neJ+1ZZsde1SLL3TNy/QQJZV1F0ua2rBZZe4/VFoQbU+O8VhN7nYvwdNlLzuqpWlxyMX4De9lZ437stotJ2R7SX+6ZglzqOj0eD/GLYnt0LI1i1p60J3SxDuWL4kgrfdLPmWzFvKf4RiB2RrmgLnHOSP44xA2nOLzpOES3nRly4zjtUtSRkw+IPjlH7bNKdFH887cr0cmrszvCBlg5hgvkNiuvtChwWwTs9tzdX7NnzzdGdSpiD9gF2FhkJHmCuaTh1Or01i+K1bdJ4dWz7ABLk+XLjQDX2lwTVrt2buQX7b2YRd4lZjwt3no5Eoxf+fbybhSwl7eEnR/ZZofsc0J6/aZr558H4xG97WGiBD7xyaWM59l/vVFA9q0sZ5FMl+Cf9WhWrlwLSXLNn08rv1z9d8Yn5bpN7ZCUL2CjgAxZ+eWq93znl+6qaZEk+ebPhJVfvv75OmYluyZqni0XkNcqv1z91+ekXMuiZln3XarLlKqeDfL3WWmyzZ9bqgjZ5te5tkTNsO77upcpV39sbWuXJKnmz/GUX6r+9yelKimvfrkAPlfEDMD5WPy1admSlcC69LqVXyeQNQ9I4F2cx3czA8az7MtNssVbgXXuiO7ICqW9f4/wOKdZo0rwn7DthXM2x1ECtVGuNT+LcjmV6dmtRqvPYE1tGO/100Srirejptv/5v3e3+q3hpe9j31c0D/i2B8d3ZTsK75Ip3FOO9JrX8VuPtvK50M+bT7yRX4RqrqkPzADRcajlg/bzbl1T86783xIzzci7HaDxbKb9Cppm3RkYiN30TNvR334C/oC+y5X4MccYKfGE/b3696mr1/h/471+3t4tG8Ro809lZ/DnqwzRA9cmV0QfURoJvubDWl7jX3RnoeIm7c0Dev5fcMf62Wj8N7kXuoi/naxz+z5+0T+Xq59p1radyamxOWOOFpP37mJLLDb8LlvzfSgZAlH+EvvC4iwm0/ntOGeEvfW5JxTQu165ukDen5H6VxYJ7hXx3lcTpwZGJ/blVZeH8jyuead0Y6X65Enqu05anx856x3zfOPHVlrdkwt/0IvHlffIV7sccL36bKWpa9ZXJPZ9VLXd4ge2E69936iXg2s4g+j/pzFH7uZwROfNW7DxfObxxtrH1HF/Hr5+//nlZiFr7j3oMeJueUK2stgTg+MIzx5FL1R+gzdzwv1rsm7mvSud1cnnzl/Lo13UvcRuakzBDRNXP3twrqUTCakJ6V3Vz2ayJjIxPRepvTS9Pt4t0tvNOcl2Wv6fVXEYD2p2IrNd/2t/g8TSSHkoCsBAA=='
databytes = base64.b64decode(datastr.encode())
dataunzipped = gzip.decompress(databytes)
finalstr = dataunzipped.decode('utf-8')
print(finalstr[::2])

{
    "$id": 0,
    "$type": "0|DiskCardGame.AscensionSaveData, Assembly-CSharp",
    "dialogueData": {
        "$id": 1,
        "$type": "1|DiskCardGame.DialogueEventsData, Assembly-CSharp",
        "eventData": {
            "$id": 2,
            "$type": "2|System.Collections.Generic.List`1[[DiskCardGame.DialogueEventsData+EventSaveData, Assembly-CSharp]], mscorlib",
            "$rlength": 133,
            "$rcontent": [
                {
                    "$id": 3,
                    "$type": "3|DiskCardGame.DialogueEventsData+EventSaveData, Assembly-CSharp",
                    "eventId": "P03FinalShopNode",
                    "repeatCount": 11
                },
                {
                    "$id": 4,
                    "$type": 3,
                    "eventId": "P03AscensionPreIntro",
                    "repeatCount": 5
                },
                {
                    "$id": 5,
                    "$type": 3,
                    "eventId": "P03AboutMe",
 

In [2]:
client = p03_wiki.login()

In [3]:
wiki_strs = {
    "base": "==Vanilla Sigils==\n",
    "new": "==New Sigils=="
}

for k in wiki_strs:
    wiki_strs[k] += """{| class="wikitable" style="width:90%; align:center; text-align:center"
|-
! Icon
! Power Level
! Name
! Description"""

for fname in glob.glob(os.path.join(os.environ["INSCRYPTION_PATH"], 'cardexports', "ability*.json")):
    with open(fname) as f:
        abjson = json.load(f)

    k = "base" if int(abjson["ability"]) < 1000 else "new"

    if k == "base" and 2 not in abjson["metaCategories"]:
        continue

    wiki_strs[k] += "\n|-\n"
    wiki_strs[k] += "| [[File:Ability_" + "_".join(abjson["rulebookName"].split(" ")) + ".png|75px]]\n"
    wiki_strs[k] += "| " + str(abjson["powerLevel"]) + "\n"
    wiki_strs[k] += "| [[" + abjson["rulebookName"] + "]]\n"
    wiki_strs[k] += "| " + abjson["rulebookDescription"].replace("[creature]", "a card bearing this sigil")

for k in wiki_strs:
    wiki_strs[k] += "\n|}"

In [4]:
print(wiki_strs["new"])

==New Sigils=={| class="wikitable" style="width:90%; align:center; text-align:center"
|-
! Icon
! Power Level
! Name
! Description
|-
| [[File:Ability_Acceptable_Sacrifice.png|75px]]
| 2
| [[Acceptable Sacrifice]]
| Sacrificing a card bearing this sigil will count as if two creatures were sacrificed
|-
| [[File:Ability_Airborne_Conduit.png|75px]]
| 3
| [[Airborne Conduit]]
| Cards within a circuit completed by a card bearing this sigil have Airborne.
|-
| [[File:Ability_Airborne_Latch.png|75px]]
| 1
| [[Airborne Latch]]
| When a card bearing this sigil perishes, its owner chooses a creature to gain the Airborne sigil.
|-
| [[File:Ability_Alarm_Clock.png|75px]]
| 1
| [[Alarm Clock]]
| The creature that this sigil is pointing to will gain +1 attack. The clock will turn at the beginning of the player's turn.
|-
| [[File:Ability_Annoying_Latch.png|75px]]
| 1
| [[Annoying Latch]]
| When a card bearing this sigil perishes, its owner chooses a creature to gain the Annoying sigil.
|-
| [[File:

In [2]:
from PIL import Image
import numpy as np

In [16]:
im = Image.open(os.path.abspath(os.path.join("..", "P03ExpansionPack3", "assets", "portrait_big_blood_card.png")))

In [17]:
ima = np.array(im.convert('RGBA'))
ima.shape

(94, 114, 4)

In [18]:
ima[:,:,3] = np.where(
    ima[:,:,3] < 40, 0, ima[:,:,3]
)
ima[:,:,3] = np.where(
    (ima[:,:,3] >= 40) & (ima[:,:,3] < 200), 128, ima[:,:,3]
)
ima[:,:,3] = np.where(
    (ima[:,:,3] >= 200), 255, ima[:,:,3]
)

In [19]:
im2 = Image.fromarray(ima)
im2.save(os.path.abspath(os.path.join("..", "P03ExpansionPack3", "assets", "portrait_big_blood_card_2.png")))

In [17]:
for fname in glob.glob(os.path.join(os.environ["INSCRYPTION_PATH"], 'cardexports', "p03face*.png")):
    im = Image.open(fname)
    im2 = im.crop((439,149,439+973,149+692))
    im2.save(os.path.join('cardexports', os.path.basename(fname)))

In [6]:
for fname in glob.glob(os.path.join(os.environ["INSCRYPTION_PATH"], 'cardexports', "item*.png")):
    im = Image.open(fname)
    im2 = im.crop((680,178,680+557,178+733))
    im3 = im2.crop(im2.getbbox())
    im3.save(os.path.join('cardexports', os.path.basename(fname).replace("zorro.inscryption.infiniscryption.p03kayceerun_", "")))

In [52]:
#@functools.cache
def _get_eyes(fname='C:\\GIT\\Inscryption-Kaycee\\globalgamemanagers\\Assets\\Resources\\art\\ui\\ascension\\ascensionicon_activated_default.png'):
    eyes = Image.open(fname)
    eyes = eyes.convert('RGBA')

    eyes = np.array(eyes)
    r, g, b, a = eyes.T
    eyes = a > 0
    print(fname)
    return eyes

def get_eyes(fname):
    for suffix in ['_activated', '_active', 'activated', 'active']:
        try:
            return _get_eyes(fname.replace('.png', f'{suffix}.png'))
        except:
            continue
    try:
        return _get_eyes(fname.replace('ascensionicon', 'ascensionicon_activated'))
    except:
        pass
    return _get_eyes()

In [53]:
dirs = [
    os.path.abspath(os.path.join('..', 'P03KayceeRun', 'assets')),
    'C:\\GIT\Inscryption-Kaycee\\globalgamemanagers\\Assets\\Resources\\art\\ui\\ascension'
]
for dir in dirs:
    for fname in glob.glob(os.path.abspath(os.path.join(dir, 'ascensionicon*.png'))):
        if 'activate' in fname:
            continue

        print(fname)

        im = Image.open(fname)
        im = im.convert('RGBA')

        data = np.array(im)
        r, g, b, a = data.T

        new_r = a > 0
        data[..., :][new_r.T] = (155, 37, 49, 255)

        eyes = get_eyes(fname)
        data[..., :][eyes.T] = (247, 248, 206, 255)

        im2 = Image.fromarray(data)
        im2.save(os.path.join('cardexports', os.path.basename(fname)))

c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_bomb.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_bombactivated.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_bounthunter.png
C:\GIT\Inscryption-Kaycee\globalgamemanagers\Assets\Resources\art\ui\ascension\ascensionicon_activated_default.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_broken_bridge.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_broken_bridge_activated.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_conveyorbattle.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_conveyorbattle_active.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_conveyorbattle_active.png
C:\GIT\Inscryption-Kaycee\globalgamemanagers\Assets\Resources\art\ui\ascension\ascensionicon_activated_default.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_eccentricpainter.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon

<>:3: DeprecationWarning: invalid escape sequence '\I'
<>:3: DeprecationWarning: invalid escape sequence '\I'
C:\Users\divis\AppData\Local\Temp\ipykernel_6564\1142402360.py:3: DeprecationWarning: invalid escape sequence '\I'
  'C:\\GIT\Inscryption-Kaycee\\globalgamemanagers\\Assets\\Resources\\art\\ui\\ascension'


In [26]:
site = p03_wiki.login()

In [32]:
fc = site.pages["Flamecharmer"]
fc.text()

'{{CardBox\n| name = Flamecharmer\n| id = P03KCMXP2_FlameCharmer\n| artist = \n| expansion = Expansion Pack #2\n| cost = 2 Energy\n| attack = 0\n| health = 3\n| sigils = {{Pagelist|Flame Stoker|Stone Giver}}\n| quality = Unobtainable\n| region = Unobtainable\n}}\n\n== Summary ==\n\nFlamecharmer is a card from [[Expansion Pack 2]]. \n\n== Strategy ==\n\nThis section is currently blank.\n\n== Trivia ==\n\nThis section is currently blank.\n\n[[Category:Cards]]'

In [39]:
unob_cards = list(p.name for p in site.categories["Unobtainable Cards"])
xp2_cards = list(p for p in site.categories["Expansion Pack 2 Cards"] if p.name not in unob_cards)

In [50]:
outtext = ''
#xp2_cards = list(p for p in site.categories["Base Game Cards"] if p.name not in unob_cards)
for p in site.categories["Unobtainable Cards"]:
    text = p.text()
    fname = [l.replace('| id = ', '') for l in text.splitlines() if "id = " in l][0]
    outtext += f'File:{fname}.png|[[{p.name}]]\n'
print(outtext)

File:CXformerElk.png|[[3LK]]
File:P03KCM_CXformerAlpha.png|[[41PH4]]
File:P03KCM_BOUNTY_SPAWNER.png|[[Activated Hunter]]
File:CXformerAdder.png|[[ADD3R]]
File:Tree_Hologram.png|[[Ancient Fir]]
File:AnnoyTower.png|[[Annoy FM]]
File:P03KCMXP1_MantisBot.png|[[Asmanteus]]
File:P03KCMXP1_WolfBot.png|[[B30WULF]]
File:P03KCMXP1_BeastMaster2.png|[[B3A5T GR4ND M4ST3R]]
File:Angler_Fish_Bad.png|[[Bad Fish]]
File:P03KCM_Draft_Token.png|[[Basic Token]]
File:P03KCM_Blockchain.png|[[Blockchain]]
File:BridgeRailing.png|[[Bridge Rails]]
File:BrokenBot.png|[[Broken Bot]]
File:P03KCM_CODE_BUG.png|[[Bug]]
File:CaptiveFile.png|[[Captive File]]
File:P03KCMXP2_CHARGE.png|[[Charge!]]
File:P03KCM_CODE_BLOCK.png|[[Code Snippet]]
File:ConduitTower.png|[[Conduit Tower]]
File:P03KCM_AboveCurve2.png|[[Curve Hopper Hopper]]
File:DeadTree.png|[[Dead Tree]]
File:EmptyVessel.png|[[Empty Vessel]]
File:P03KCM_MYCO_CONSTRUCT_BASE.png|[[Experiment]]
File:P03KCM_FAILED_EXPERIMENT.png|[[Failed Experiment]]
File:P03KCM_FIREW

In [46]:
[l.replace('| id = ', '') for l in test_t.splitlines() if "id = " in l][0]

'P03KCMXP1_RoboAngel'

In [36]:
category = site.categories['Cards']
for page in category:
    try:
        text = page.text()
        edit = False
        if "Expansion Pack #1" in text:
            text = text.replace("Expansion Pack #1", "Expansion Pack 1")
            edit = True
        if "Expansion Pack #2" in text:
            text = text.replace("Expansion Pack #2", "Expansion Pack 2")
            edit = True
        if edit:
            print(f"Editing page {page}")
            page.edit(text)
        else:
            print(f"Touching page {page}")
            page.touch()
    except:
        print("Taking a break")
        time.sleep(30)
        page.touch()

Touching page <Page object 'b'3leph4nt'' for <Site object 'p03kcmod.wiki.gg/'>>
Touching page <Page object 'b'3LK'' for <Site object 'p03kcmod.wiki.gg/'>>
Touching page <Page object 'b'41PH4'' for <Site object 'p03kcmod.wiki.gg/'>>
Touching page <Page object 'b'42415454455259'' for <Site object 'p03kcmod.wiki.gg/'>>
Touching page <Page object 'b'49er'' for <Site object 'p03kcmod.wiki.gg/'>>
Touching page <Page object 'b'4D4952524F52'' for <Site object 'p03kcmod.wiki.gg/'>>
Touching page <Page object 'b'656D6572616C64'' for <Site object 'p03kcmod.wiki.gg/'>>
Touching page <Page object 'b'Activated Hunter'' for <Site object 'p03kcmod.wiki.gg/'>>
Touching page <Page object 'b'ADD3R'' for <Site object 'p03kcmod.wiki.gg/'>>
Touching page <Page object 'b'Airspace Conduit'' for <Site object 'p03kcmod.wiki.gg/'>>
Touching page <Page object 'b'Alarm Bot'' for <Site object 'p03kcmod.wiki.gg/'>>
Touching page <Page object 'b'AmmoBot'' for <Site object 'p03kcmod.wiki.gg/'>>
Touching page <Page obj

In [37]:
page.name

'Zombie Process'

In [20]:
for fname in glob.glob(os.path.abspath(os.path.join('cardexports', 'generator-cards-export-errorsfixed', '*.png'))):
    try:
        site.Pages[f"File:{os.path.basename(fname)}"].delete()
    except:
        continue

In [21]:
for fname in glob.glob(os.path.abspath(os.path.join('cardexports', 'generator-cards-export-errorsfixed', '*.png'))):
    if fname.lower() == "techmoxtriple" or "sentinel" in fname.lower():
        try:
            site.Pages[f"File:{os.path.basename(fname)}"].delete()
        except:
            continue